In [2]:



#from pytorch_lightning.callbacks import TQDMProgressBar
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from models.transformer.transformer_main import MyTransformer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from ray import tune
from ray.tune.integration.pytorch_lightning import TuneReportCheckpointCallback

from models.transformer.my_functrions import make_dataset, get_torch_data_loaders, RMSELoss


In [4]:
torch.__version__

'1.13.1+cu117'

In [5]:
torch.manual_seed(0)
np.random.seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device



device(type='cpu')

In [7]:
df = pd.read_csv(
    '../datasets/prepared/btc/ta_corr_01_fd.csv',
    parse_dates=True
    )
target_col = 'log_returns'
df = df.set_index(['timestamp'])
df.index = pd.to_datetime(df.index)
exclude_cols = ['log_close', 'Close']
df

,log_close,Close,log_returns,diff_fear_greed,volatility_kcp,volume_em,volume_vpt,momentum_stoch_rsi,volatility_bbp,momentum_stoch,...,trend_macd_diff,trend_stc,volume_sma_em,trend_aroon_ind,momentum_tsi,trend_aroon_down,trend_vortex_ind_neg,trend_psar_down_indicator,trend_adx_neg,volatility_bbli
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-11-27,5.912611,369.670013,0.003523,-0.006765,0.501327,-40.468161,-3.149466e+05,0.388997,0.425509,34.013853,...,-0.855557,24.373770,-502.853674,36.0,-0.527280,4.0,1.288202,0.0,20.285280,0.0
2014-11-28,5.930777,376.446991,0.018166,0.007878,0.726950,-52.121651,4.515396e+05,0.748693,0.490535,48.927911,...,-0.354460,46.507056,-281.225092,32.0,0.152113,4.0,1.133492,0.0,22.078843,0.0
2014-11-29,5.928234,375.490997,-0.002543,-0.012831,0.700354,927.476194,3.816202e+05,0.714180,0.469128,47.481343,...,-0.099079,68.204734,0.631289,28.0,0.610003,4.0,1.070125,0.0,20.806231,0.0
2014-11-30,5.935019,378.046997,0.006784,-0.003504,0.764513,-196.001510,2.354075e+04,0.813134,0.496233,51.348971,...,0.213258,84.102367,40.733570,20.0,1.277897,8.0,1.032653,0.0,20.063468,0.0
2014-12-01,5.938182,379.244995,0.003164,-0.007124,0.718714,133.611388,9.986327e+04,1.000000,0.504775,69.447910,...,0.457770,92.051184,-12.443357,20.0,1.977937,4.0,1.119928,0.0,19.494893,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-29,10.230843,27745.884766,-0.012171,0.987829,1.050246,2858.497078,4.753301e+08,0.811280,0.932253,73.001414,...,142.801589,75.000019,451.343496,-76.0,-8.043034,80.0,0.918640,0.0,18.152938,0.0
2023-05-30,10.229273,27702.349609,-0.001570,-0.001570,0.955887,-624.340933,-2.044453e+08,0.787049,0.897492,71.288406,...,158.821013,87.500010,434.361789,-72.0,-5.906286,76.0,0.923086,0.0,17.383497,0.0
2023-05-31,10.211695,27219.658203,-0.017578,-1.017578,0.580642,-2882.568661,-2.935917e+08,0.526399,0.601630,52.295615,...,132.467667,93.750005,251.374776,-68.0,-5.567434,72.0,0.962587,0.0,22.440350,0.0


In [15]:
# config = {
#     'batch_size': tune.grid_search([4]),
#     'num_layers': tune.grid_search([4]),
#     'dropout': tune.grid_search([0.2]),
#     'nhead': tune.grid_search([1]),
#     #'attn_type': tune.grid_search([ 'dense', '', 'rv_mix']),
#     #'attn_type': tune.grid_search(['', 'fac_dense', 'dense', 'rv_mix', 'dv_mix']),
#     'attn_type': tune.grid_search(['fac_dense',]),
#     'learning_rate': tune.grid_search([1e-5]),
#     'weight_decay': tune.grid_search([1e-6]),
#     'patience': tune.grid_search([50]),
#     'n_epochs': tune.grid_search([5]),
#     'timestep': tune.grid_search([30]),
#     'horizon': tune.grid_search([21]),
#     'ntest': tune.grid_search([21]),
#     'model_checkpoint_outputdir': 'multioutput/ta_corr_01_fd2'
# }
config = {
    'batch_size': tune.choice([4, 8]),
    'num_layers': tune.choice([4, 8]),
    'dropout': tune.choice([0.2]),
    'nhead': tune.choice([1, 4, 6]),
    #'attn_type': tune.grid_search([ 'dense', '', 'rv_mix']),
    #'attn_type': tune.grid_search(['', 'fac_dense', 'dense', 'rv_mix', 'dv_mix']),
    'attn_type': tune.choice(['fac_dense', '']),
    'learning_rate': tune.choice([1e-5]),
    'weight_decay': tune.choice([1e-6]),
    'patience': tune.choice([25]),
    'n_epochs': tune.choice([50]),
    'timestep': tune.choice([10, 20, 30]),
    'horizon': tune.choice([21]),
    'ntest': tune.choice([42]),
    'model_checkpoint_outputdir': 'multioutput/ta_corr_01_fd3'
}


In [13]:


def train_transformer(config):
    print('CUDA', torch.cuda.is_available())
    timestep= config['timestep']
    ntest = config['ntest']
    horizon = config['horizon']
    decoder_size = 16

    Xtrain, Ytrain, Xtest, Ytest, XVal, YVal, scaler = make_dataset(
        df, target_col='log_returns', exclude_cols=exclude_cols, timestep=timestep, ntest=ntest, horizon=horizon
    )
    train_loader, val_loader, test_loader, test_loader_one = get_torch_data_loaders(
        Xtrain, Ytrain, Xtest, Ytest, XVal, YVal, config['batch_size']
    )
    feature_size = Xtrain.shape[-1] #input_dim

    loss_fn = RMSELoss

    # loss_fn = RMSELoss
    # # train
    
    model = MyTransformer(
        loss_fn=loss_fn,
        batch_size=config['batch_size'],
        decoder_size=decoder_size, 
        timestep=timestep,
        horizon=horizon,
        feature_size=feature_size,
        num_layers=config['num_layers'],
        dropout=config['dropout'],
        nhead=config['nhead'],
        attn_type=config['attn_type'],
        learning_rate=config['learning_rate'],
        weight_decay=config['weight_decay']
    )


    early_stop_callback = EarlyStopping(monitor="val_loss", patience=config['patience'], verbose=False, mode="min")

    filename = f"{config['attn_type']}_timestep={timestep}" + '{epoch}-{val_loss:.3f}'
    checkpoint_callback = ModelCheckpoint(
        dirpath=f"./checkpoints/{config['model_checkpoint_outputdir']}",
        filename=filename,
        save_top_k=1, 
        monitor="val_loss"
    )
    
    tune_report_callback = TuneReportCheckpointCallback(
        {"loss": "val_loss"},
        on="validation_end"
    )

    trainer = pl.Trainer(
        callbacks=[early_stop_callback,checkpoint_callback, tune_report_callback], 
        max_epochs=config['n_epochs'],
        logger=False,
    )

    trainer.fit(model, train_loader, val_loader)

    #=========================================================================
    #TODO SAVE LOGS TO GOOGLE DRIVE
    # dont save all checkpoints!
    # add search algrothitm!
    #=========================================================================

In [14]:
# analysis = tune.run(
#     tune.with_parameters(train_transformer),
#     config=config,
#     num_samples=10,
#     resources_per_trial={"cpu": 1, "gpu": 1},
#  )

trainable = tune.with_resources(
    train_transformer,
    {
        "cpu": 4,
        #"gpu": 1,
        #'accelerator': NVIDIA_TESLA_A100
    }
)

analysis = tune.run(
    trainable,
    # resources_per_trial={
    #     "cpu": 12,
    #     "gpu": 1,
    #     'accelerator_type': NVIDIA_TESLA_A100
    # },
    metric="loss",
    mode="min",
    config=config,
    #num_samples=4,
    name="tune_transformer",
    keep_checkpoints_num=3,
    local_dir=f"./checkpoints/{config['model_checkpoint_outputdir']}"
)

2023-06-02 19:48:11,613	INFO worker.py:1625 -- Started a local Ray instance.
2023-06-02 19:48:12,449	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
/home/olexandr/myenvs2/diploma1/lib/python3.9/site-packages/ray/tune/experiment/experiment.py:170: UserWarning:

The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.



(train_transformer pid=11571) CUDA False
(train_transformer pid=11571) X.shape (3060, 30, 36) Y.shape (3060, 21)
(train_transformer pid=11571) kw_batch,feature size:  4 36
(train_transformer pid=11571) embed_dim: 36
(train_transformer pid=11571) num_heads:  1


(train_transformer pid=11571) GPU available: False, used: False
(train_transformer pid=11571) TPU available: False, using: 0 TPU cores
(train_transformer pid=11571) IPU available: False, using: 0 IPUs
(train_transformer pid=11571) HPU available: False, using: 0 HPUs
(train_transformer pid=11571) 
(train_transformer pid=11571)   | Name                | Type                    | Params
(train_transformer pid=11571) ----------------------------------------------------------------
(train_transformer pid=11571) 0 | pos_encoder         | PositionalEncoding      | 0     
(train_transformer pid=11571) 1 | encoder_layer       | TransformerEncoderLayer | 155 K 
(train_transformer pid=11571) 2 | transformer_encoder | TransformerEncoder      | 620 K 
(train_transformer pid=11571) 3 | decoder1            | Linear                  | 592   
(train_transformer pid=11571) 4 | decoder2            | Linear                  | 10.1 K
(train_transformer pid=11571) -------------------------------------------

Epoch 0:   0%|          | 0/754 [00:00<?, ?it/s]                           
(train_transformer pid=11571) training_loss 
(train_transformer pid=11571) tensor(2.2151, grad_fn=<SqrtBackward0>)
Epoch 0:   0%|          | 1/754 [00:00<01:01, 12.20it/s]
(train_transformer pid=11571) training_loss tensor(2.3022, grad_fn=<SqrtBackward0>)
Epoch 0:   3%|▎         | 26/754 [00:01<00:40, 17.89it/s]
(train_transformer pid=11571) training_loss tensor(0.7318, grad_fn=<SqrtBackward0>)
Epoch 0:   4%|▎         | 28/754 [00:01<00:40, 18.03it/s]
(train_transformer pid=11571) training_loss tensor(0.6502, grad_fn=<SqrtBackward0>)
Epoch 0:   4%|▍         | 30/754 [00:01<00:39, 18.14it/s]
(train_transformer pid=11571) training_loss tensor(0.6907, grad_fn=<SqrtBackward0>)
Epoch 0:   5%|▌         | 38/754 [00:02<00:38, 18.40it/s]
(train_transformer pid=11571) training_loss tensor(0.5228, grad_fn=<SqrtBackward0>)
Epoch 0:   5%|▌         | 40/754 [00:02<00:38, 18.47it/s]
(train_transformer pid=11571) training_los

Trial name,date,done,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_transformer_43222_00000,2023-06-02_19-48-58,False,o-dubchak-pc2,1,0.60245,192.168.248.158,11571,True,40.218,40.218,40.218,1685724538,1,43222_00000


Epoch 1:   4%|▎         | 27/754 [00:01<00:37, 19.47it/s]
(train_transformer pid=11571) training_loss tensor(0.6332, grad_fn=<SqrtBackward0>)
Epoch 1:   4%|▍         | 29/754 [00:01<00:37, 19.30it/s]
(train_transformer pid=11571) training_loss tensor(0.6219, grad_fn=<SqrtBackward0>)
Epoch 1:   5%|▍         | 34/754 [00:01<00:38, 18.94it/s]
(train_transformer pid=11571) training_loss tensor(0.3642, grad_fn=<SqrtBackward0>)
Epoch 1:   5%|▍         | 36/754 [00:01<00:38, 18.88it/s]
(train_transformer pid=11571) training_loss tensor(0.3670, grad_fn=<SqrtBackward0>)
Epoch 1:   5%|▌         | 38/754 [00:02<00:38, 18.84it/s]
(train_transformer pid=11571) training_loss tensor(0.4537, grad_fn=<SqrtBackward0>)
Epoch 1:   5%|▌         | 40/754 [00:02<00:37, 18.89it/s]
(train_transformer pid=11571) training_loss tensor(0.4504, grad_fn=<SqrtBackward0>)
Epoch 1:   6%|▋         | 48/754 [00:02<00:37, 19.01it/s]
(train_transformer pid=11571) training_loss tensor(0.7291, grad_fn=<SqrtBackward0>)
Epoch 

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir="/content/drive/MyDrive/models/transformers/checkpoints/ta_corr_with_logg_diffed"

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-05-30 19:57:36.512598: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 19:57:36.567574: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 19:57:37.483972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /usr/local/lib/python3.1

In [ ]:
analysis.

In [ ]:
%cp -av "/root/ray_results/tune_transformer" "/content/drive/MyDrive/models/transformers/checkpoints/logged_diffed_timestep30/ray_logs"

In [ ]:
analysis.get_best_config()

In [ ]:
analysis.best_checkpoint.path

'/content/drive/MyDrive/models/transformers/checkpoints/ta_corr_with_logg_diffed/tune_transformer/train_transformer_f6d79_00001_1_attn_type=dense,batch_size=4,dropout=0.2000,learning_rate=0.0000,n_epochs=70,nhead=4,num_layers=4,_2023-05-30_19-45-55/checkpoint_epoch=33-step=25942'

In [ ]:
model = MyTransformer.load_from_checkpoint(analysis.best_checkpoint.path + '/checkpoint',  loss_fn=RMSELoss)
trainer=pl.Trainer()

In [ ]:
best_config = analysis.get_best_config()

Xtrain, Ytrain, Xtest, Ytest, XVal, YVal = make_dataset(
    df, target_col='log_returns', exclude_cols=['RSI-based MA'], timestep=10, ntest=21
)
train_loader, val_loader, test_loader, test_loader_one = get_torch_data_loaders(
    Xtrain, Ytrain, Xtest, Ytest, XVal, YVal, best_config['batch_size']
)

In [ ]:
trainer.test(model, test_loader_one)

In [16]:
import ray

context = ray.init()